# 1. Importar tablas

In [1]:
/ 1.1. Importar tabla empleados
employees:get `:data/employees;

/show employees;

In [22]:
/ 1.2. Importar tabla de las bases de contribución
contributionBasis:get `:data/contributionBasis;

/show employees

In [3]:
/ 1.3. Importar tabla de tipos
rateTypes:("DDFSsS";enlist csv) 0: `$":data/rateTypes.csv";

/show rateTypes

In [4]:
/ 1.3. Importar tabla de escala irpf
irpfScale:("DDFF";enlist csv)0:`$":data/irpfScale.csv";

irpfScale: ungroup select range, rate, acc:sums prev[rate]*deltas range
                   by startDate, endDate 
                   from irpfScale;

/show irpfScale

# 2. Variables globales

In [5]:
cnaes:string asc distinct rateTypes`cnae;                                 /all the cnae's strings

# 3. Funciones


## 3.1. Cálculo de las aportaciones

In [17]:
gacnae:{`$cnaes last where cnaes in (neg til count string x)_\:string x}; /(g)et (a)ctual (cnae)

gRate:{[mo;conti;contr;cn]                                                /(g)et (rate) type
    0^first exec rate%100 
            from rateTypes 
            where contingency=conti, 
                  contributor=contr, 
                  mo within (startDate;endDate),
                  (cnae=gacnae[cn]) | cnae=`0};

## 3.2. Cálculo de las retenciones

In [18]:
calirpf:{[n;ye]                                                           /(c)alculate (irpf)
    first exec acc+rate*n-range 
          from irpfScale 
          where ye within (`year$startDate;`year$endDate), i=range bin n};

gIrpfRate:{[mo;ni;contr]                                                  /(g)et (irpf) (rate) type
    if[contr=`E;:0];
    r:first exec irpfRate 
           from irpfBasis 
           where nif=ni,year=`year$mo;
    $[r=0;0;r|0.02]};

## 3.3. Funciones para monthlyContributions

In [19]:
gconty:{[bt;ws]                                                           /(g)en (con)tribution (ty)pe
    d: `commonBase`profBase`hExtraFMBase`hExtraBase`irpfBase`meiBase!
    ((`cc;(`at;$[ws=`temp;`ded;`dei];`fp;`fgs);`hef;`he;`irpf;`mei));
    d bt};

cmonthly_contributions:{[nif;mon;contributor;conType;n;cnae]              /(c)reate monthlyContributions
        $[conType=`irpf;
          :gIrpfRate[mon;nif;contributor];
          :gRate[mon;conType;contributor;cnae]];}

# 4. Generación de las tablas

In [20]:
/---------- monthlyContributionsla auxiliar irpfBasis ----------
irpfBasis: () xkey select aIrpfBase: sum amount, mpf:5500f                /new monthlyContributionsle with: (a)ctual (irpfBase)  & "(m)ínimo (p)ersonal y (f)amiliar"
                   by year:`year$month, nif 
                   from contributionBasis
                   where baseType=`irpfBase;

update retfee: 0 | calirpf'[aIrpfBase;year] - calirpf'[mpf;year]          /add a column for (ret)ention (fee)
    from `irpfBasis;

update irpfRate: 0.01 xbar 0.005 + retfee%aIrpfBase                       /add a column for the irpf Rate
    from `irpfBasis;

meta irpfBasis

/---------- monthlyContributionsla auxiliar irpfBasis ----------
monthlyContributions: update contributionType: gconty'[baseType;workingSchedule]
                      from contributionBasis lj 
                           `nif xkey select workingSchedule, nif from employees;

monthlyContributions: raze {n: 2*count x[`contributionType];
     flip `nif`month`contributor`contributionType`amount!(
        n#x[`nif];
        n#x[`month];
        ((n div 2)#`E),(n div 2)#`T;
        (x[`contributionType]),x[`contributionType];
        n#x[`amount])} each monthlyContributions;

monthlyContributions: update amount: amount*cmonthly_contributions'[nif;month;contributor;contributionType;amount;cnae]
                      from monthlyContributions lj 
                           `nif xkey select cnae, nif from employees;

monthlyContributions: select nif, month, contributor, contributionType, amount 
                      from `monthlyContributions 
                      where amount<>0;

c        | t f a
---------| -----
year     | i   p
nif      | s    
aIrpfBase| f    
mpf      | f    
retfee   | f    
irpfRate | f    


Si se quisiese ver de forma más visual de las bases de cotización se puede pivotar la tabla de la siguiente manera:

In [21]:
show 0^exec `cc`at`fp`fgs`hef`he`irpf`mei#(contributionType!amount)
    by nif,month,contributor
    from monthlyContributions;

nif       month   contributor| cc       at       fp       fgs      hef      h..
-----------------------------| ----------------------------------------------..
00193031S 2021.07 E          | 247.8    34.65    6.3      2.1      89.52919 1..
00193031S 2021.07 T          | 49.35    0        1.05     0        14.92153 3..
00193031S 2021.08 E          | 614.611  85.94137 15.6257  5.208568 312.5141 6..
00193031S 2021.08 T          | 122.4013 0        2.604284 0        52.08568 1..
00193031S 2021.09 E          | 615.7022 86.09395 15.65344 5.217815 313.0689 6..
00193031S 2021.09 T          | 122.6186 0        2.608907 0        52.17815 1..
00193031S 2021.10 E          | 604.1195 86.2845  15.68809 5.229364 313.7618 6..
00193031S 2021.10 T          | 120.3119 0        2.614682 0        52.29364 1..
00193031S 2022.07 E          | 275.3412 38.5011  7.0002   2.3334   89.37942 1..
00193031S 2022.07 T          | 54.8349  0        1.1667   0        14.89657 3..
00193031S 2022.08 E          | 580.5155 

# 5. Guardar datos en csv

In [11]:
`:data/monthlyContributions set monthlyContributions;

In [12]:
count monthlyContributions

704396
